In [ ]:
import os
import torch
import json
import numpy as np
import pandas as pd
from PIL import Image
from open_clip import create_model_from_pretrained, get_tokenizer

# Load the model and tokenizer
model, preprocess = create_model_from_pretrained('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
tokenizer = get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

# Set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()
print(f"Model loaded on device: {device}")

In [ ]:
# Define input directories
image_folder = "../data/unified/imgs"
text_folder = "../data/unified/alt"
spec_folder = "../data/unified/specs"
output_folder = "./embeddings"
os.makedirs(output_folder, exist_ok=True)

batch_size = 50  # Adjust batch size as needed
context_length = 256

def generate_text_embeddings(folder, output_name, alt_version = None):
    if alt_version:
        folder = f"{folder}_{alt_version}"
    text_files = [f for f in os.listdir(folder) if f.lower().endswith(".txt") or f.lower().endswith(".json")]
    text_embeddings = []
    file_names = []
    
    for i in range(0, len(text_files), batch_size):
        batch_files = text_files[i:i + batch_size]
        texts = []
        
        for text_file in batch_files:
            text_path = os.path.join(folder, text_file)
            
            with open(text_path, "r", encoding="utf-8") as f:
                if text_file.lower().endswith(".json"):
                    try:
                        data = json.load(f)
                        text_desc = json.dumps(data)  # Convert JSON object to a string
                    except json.JSONDecodeError:
                        print(f"Warning: Could not parse JSON in {text_file}, skipping.")
                        continue
                else:
                    text_desc = f.read().strip()
            
            texts.append(text_desc)
            file_names.append(text_file)
        
        text_input = tokenizer(texts, context_length=context_length).to(device)
        
        with torch.no_grad():
            text_features = model.encode_text(text_input)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        
        text_embeddings.append(text_features.cpu().numpy())
        print(f"Processed {i + batch_size} of {len(text_files)} text files.")
    
    save_embeddings(text_embeddings, file_names, output_name)

# Function to generate image embeddings in batches
def generate_image_embeddings(folder, output_name):
    image_files = [f for f in os.listdir(folder) if f.lower().endswith((".png", ".jpg", ".jpeg"))]
    image_embeddings = []
    file_names = []
    
    for i in range(0, len(image_files), batch_size):
        batch_files = image_files[i:i + batch_size]
        images = []
        
        for image_file in batch_files:
            image_path = os.path.join(folder, image_file)
            image = preprocess(Image.open(image_path).convert("RGB"))
            images.append(image)
            file_names.append(image_file)
        
        images_tensor = torch.stack(images).to(device)
        
        with torch.no_grad():
            image_features = model.encode_image(images_tensor)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        
        image_embeddings.append(image_features.cpu().numpy())
        print(f"Processed {i + batch_size} of {len(image_files)} image files.")
    
    save_embeddings(image_embeddings, file_names, output_name)

# Function to save embeddings as TSV and Parquet
def save_embeddings(embeddings, file_names, output_name):
    if not embeddings:
        print(f"No embeddings generated for {output_name}")
        return
    
    embeddings_np = np.vstack(embeddings)
    df = pd.DataFrame(embeddings_np, columns=[f"dim_{i}" for i in range(embeddings_np.shape[1])])
    df.insert(0, "file_name", file_names)
    
    tsv_path = os.path.join(output_folder, f"{output_name}.tsv")
    parquet_path = os.path.join(output_folder, f"{output_name}.parquet")
    
    df.to_csv(tsv_path, sep="\t", index=False)
    df.to_parquet(parquet_path, index=False)
    
    print(f"Embeddings saved: {tsv_path} and {parquet_path}")

In [ ]:
# Generate embeddings
print("Generating text embeddings...")
generate_text_embeddings(text_folder, "text_embeddings", alt_version="0_2_2")

print("Generating specification embeddings...")
generate_text_embeddings(spec_folder, "spec_embeddings")

print("Generating image embeddings...")
generate_image_embeddings(image_folder, "image_embeddings")

print("All embeddings generated and saved successfully!")

In [ ]:
# Generate embeddings 0_2_4
print("Generating text embeddings...")
generate_text_embeddings(text_folder, "text_0_2_4_embeddings", alt_version="0_2_4")

In [ ]:
# Generate embeddings 0_2_4_llm_fs_single
print("Generating text embeddings...")
generate_text_embeddings(text_folder, "text_0_2_4_llm_fs_single_embeddings", alt_version="0_2_4_llm_fs_single")

In [ ]:
import pandas as pd

# Load the TSV file
tsv_path = "./embeddings/image_embeddings.tsv"
df_tsv = pd.read_csv(tsv_path, sep="\t")

# Display the first few rows
print(df_tsv.head())

In [ ]:
# Load the Parquet file
parquet_path = "./embeddings/spec_frequency.parquet"
df_parquet = pd.read_parquet(parquet_path)

# Display the first few rows
print(df_parquet.head())

In [ ]:
# Convert TSV to parquet

import pandas as pd

# Define file paths
tsv_path = "./embeddings/spec_frequency.tsv"
parquet_path = "./embeddings/spec_onehot.parquet"

# Read the TSV file
df = pd.read_csv(tsv_path, sep="\t")

# Save as Parquet file
df.to_parquet(parquet_path, index=False)

print(f"Converted {tsv_path} to {parquet_path}")